In [ ]:
# load things
%matplotlib inline

from pyCDFTOOLS.cdfmocsig import *

import numpy as np
import glob
import netCDF4, copy

In [ ]:
# load data and do relevant processing on the native grid

data_dir = "/home/julian/data/NEMO_data/eORCA1-LIM3/default/"

# grab the MOC_V list
file_list = []
for file in glob.glob(data_dir + "*MOC_V*.nc"):
    file_list.append(file)

# sort it according to the timestamps
file_list.sort()

# generate bins from presets (available for 0, 1000, 2000m depth), or define it in a dictionary as
#    bins = {"pref"   : 2000,
#            "nbins"  : 158,
#            "sigmin" : 30.0,
#            "sigstp" : 0.05}

bins = sigma_bins(2000)

# for putting extra options in
#   -- kt       = number for using a specified time entry (python indexing)
#   -- kz       = number for using a specified vertical level/layer (python indexing)
#   -- lprint   = True   for printing out variable names in netcdf file
#   -- lverb    = True   for printing out more information
#   -- lg_vvl   = True   for using s-coord (time-varying metric)
#   -- ldec     = True   decompose the MOC into some components
#   -- leiv     = True   for adding the eddy induced velocity component
#        eivv_var = string for EIV-v variable name
#   -- lisodep  = True   (not yet implemented) output zonal averaged isopycnal depth
#   -- lntr     = True   (not yet implemented) do binning with neutral density
kwargs =  {"kt"     : 0,
           "lprint" : False,
           "lg_vvl" : False,
           "ldec"   : False,
           "leiv"   : True,  "eivv_var" : "voce_eiv",
           "lisodep": False,
           "lntr"   : False}

In [ ]:
# cycle through the file lists and compile
print("%g files found, cycling through them..." % len(file_list))

for i in range(len(file_list)):
    fileV = file_list[i].replace(data_dir, "") # strip out the data_dir
    fileT = fileV.replace("_V_", "_T_") # replace V with T
    print(" ")
    print("working in file = %g / %g" % (i + 1, len(file_list)))
    if i == 0:
        sigma, latV, dmoc_temp, opt_dic = cdfmocsig_tave(data_dir, fileV, "voce", fileT, "toce", "soce", bins, **kwargs)
        dmoc = dmoc_temp / len(file_list)
    else:
        _, _, dmoc_temp, _ = cdfmocsig_tave(data_dir, fileV, "voce", fileT, "toce", "soce", bins, **kwargs)
        dmoc += dmoc_temp / len(file_list)
        
print(" ")
print("returning final time-averaged field")

In [ ]:
# save it if need be

# open a new netCDF file for writing.
ncfile = netCDF4.Dataset("moc_tave_testing.nc", "w", format = "NETCDF4") 
ncfile.title = "diagnosed MOC in density co-ordinates"

# create the dimensions.
ncfile.createDimension("sigma", sigma.shape[0])
ncfile.createDimension("y", latV.shape[0])
ncfile.createDimension("time", len(np.asarray([0.0])))

# first argument is name of variable, 
# second is datatype,
# third is a tuple with the names of dimensions.

lat_netcdf = ncfile.createVariable("latV", np.dtype("float32").char, "y")
lat_netcdf[:] = latV
lat_netcdf.units = "deg"
lat_netcdf.long_name = "y"

sigma_netcdf = ncfile.createVariable("sigma", np.dtype("float32").char, "sigma")
sigma_netcdf[:] = sigma
sigma_netcdf.units = "kg m-3"
sigma_netcdf.long_name = "sigma"

# global moc
moc_glob_tave_netcdf = ncfile.createVariable("rmoc_glob_tave", np.dtype("float32").char, 
                                            ("time", "sigma", "y"), fill_value = 1e20)
moc_glob_tave_netcdf[:] = dmoc
moc_glob_tave_netcdf.units = "Sv"
moc_glob_tave_netcdf.long_name = "RMOC in density co-ordinates"

# add the other MOCs and variables here after figuring out how to split them

# close the file.
ncfile.close()

print("*** SUCCESS writing example file! ***")
